In [47]:
%matplotlib inline

import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.externals import joblib
from tqdm import tqdm_notebook as tqdm
import warnings

warnings.filterwarnings('ignore')

DIR = '../data/raw/'
NROWS = None

In [48]:
bureau = pd.read_csv(os.path.join(DIR,'bureau.csv.zip'), nrows=NROWS)
display(bureau.head())

display(bureau.shape)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


(1716428, 17)

Its considerably larger than the application table with 1.7M rows but only 17 columns.
Contains `SK_ID_CURR` main unique id as well bureau table id `SK_ID_BUREAU`.
We can see dataset with 300k rows and 122 columns. Not everything is a feature though.

# Features

## Columns
Let's see which columns can be used as features.

In [49]:
desc = pd.read_csv(os.path.join(DIR, 'HomeCredit_columns_description.csv'), encoding='latin-1')

desc[(desc['Row'].isin(bureau.columns)) & 
     (desc['Table']=='bureau.csv')][['Row','Description','Special']]

,Row,Description,Special
122,SK_ID_CURR,ID of loan in our sample - one loan in our sam...,hashed
124,CREDIT_ACTIVE,Status of the Credit Bureau (CB) reported credits,NaN
125,CREDIT_CURRENCY,Recoded currency of the Credit Bureau credit,recoded
126,DAYS_CREDIT,How many days before current application did c...,time only relative to the application
127,CREDIT_DAY_OVERDUE,Number of days past due on CB credit at the ti...,NaN
128,DAYS_CREDIT_ENDDATE,Remaining duration of CB credit (in days) at t...,time only relative to the application
129,DAYS_ENDDATE_FACT,Days since CB credit ended at the time of appl...,time only relative to the application
130,AMT_CREDIT_MAX_OVERDUE,Maximal amount overdue on the Credit Bureau cr...,NaN
131,CNT_CREDIT_PROLONG,How many times was the Credit Bureau credit pr...,NaN
132,AMT_CREDIT_SUM,Current credit amount for the Credit Bureau cr...,NaN


In this table we can find information on past loans from a particular person.
It helps us see how many loans people took and how they paid them off.

Let's take a look at all the records from one example person:

In [50]:
bureau[bureau['SK_ID_CURR']==215354]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.00,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,NaN
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.0,0,42103.8,42103.80,0.00,0.0,Consumer credit,-22,NaN
225157,215354,5714458,Closed,currency 1,-1872,0,-776.0,-1203.0,NaN,0,494550.0,0.00,0.00,0.0,Consumer credit,-696,NaN
225158,215354,5714459,Closed,currency 1,-1734,0,-638.0,-1336.0,NaN,0,1084032.0,0.00,NaN,0.0,Car loan,-1336,NaN
225159,215354,5714460,Closed,currency 1,-1333,0,-1089.0,-987.0,NaN,0,102150.0,NaN,NaN,0.0,Consumer credit,-984,NaN


We can extract features like total load amount, number of past loans.

Since we will be calculating features on the groupby object (based on unique SK_ID_CURR) it is 
helpful to create an empty **features** object and add new features to it.

In [58]:
groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])
groupby_features = []
features = pd.DataFrame({'SK_ID_CURR':bureau['SK_ID_CURR'].unique()})
features.head()

,SK_ID_CURR
0,215354
1,162297
2,402440
3,238881
4,222183


How many past loans a person had.

In [59]:
group_obj = groupby_SK_ID_CURR['DAYS_CREDIT'].agg('count').\
  reset_index().rename(index=str, columns={'DAYS_CREDIT': 'bureau_number_of_past_loans'})
groupby_features.append(group_obj)

How many different types of credit.

In [60]:
group_obj = groupby_SK_ID_CURR['CREDIT_TYPE'].agg('nunique').reset_index().\
  rename(index=str, columns={'CREDIT_TYPE': 'bureau_number_of_loan_types'})
groupby_features.append(group_obj)

Total debt and total credit.

In [61]:
group_obj = groupby_SK_ID_CURR['AMT_CREDIT_SUM_DEBT'].agg('sum').reset_index().\
  rename(index=str, columns={'AMT_CREDIT_SUM_DEBT': 'bureau_total_customer_debt'})
groupby_features.append(group_obj)

group_obj = groupby_SK_ID_CURR['AMT_CREDIT_SUM'].agg('sum').reset_index().\
  rename(index=str, columns={'AMT_CREDIT_SUM': 'bureau_total_customer_credit'})
groupby_features.append(group_obj)

Total overdue amount.

In [62]:
group_obj = groupby_SK_ID_CURR['AMT_CREDIT_SUM_OVERDUE'].agg('sum').reset_index().\
   rename(index=str, columns={'AMT_CREDIT_SUM_OVERDUE': 'bureau_total_customer_overdue'})
groupby_features.append(group_obj)

In [63]:
for group_obj in groupby_features:
    features = features.merge(group_obj, on=['SK_ID_CURR'], how='left')
features.head()

,SK_ID_CURR,bureau_number_of_past_loans,bureau_number_of_loan_types,bureau_total_customer_debt,bureau_total_customer_credit,bureau_total_customer_overdue
0,215354,11,3,284463.18,5973945.30,0.0
1,162297,6,3,0.00,8230386.15,0.0
2,402440,1,1,76905.00,89910.00,0.0
3,238881,8,2,552730.50,1285239.06,0.0
4,222183,8,3,1185081.84,7158960.00,0.0


In [67]:
NUMERICAL_COLUMNS = ['bureau_number_of_past_loans',
 'bureau_number_of_loan_types',
 'bureau_total_customer_debt',
 'bureau_total_customer_credit',
 'bureau_total_customer_overdue']